In [26]:
import pandas as pd
# import sqlalchemy
from sqlalchemy import create_engine
import urllib.parse

import configparser
# import os
from pathlib import Path
import numpy as np

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from neo4j import GraphDatabase

import datetime
import os
from typing import Dict
import traceback

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
config_file_path = './config.ini'

if not Path(config_file_path).exists():
    logger.warning(f"Config file {config_file_path} not found!")

config = configparser.ConfigParser()
config.read(config_file_path)

if 'DATABASE' not in config:
    raise ValueError("DATABASE section not found in config")

db_config = {
    'host': config['DATABASE']['host'],
    'port': int(config['DATABASE']['port']),
    'username': config['DATABASE']['username'],
    'password': config['DATABASE']['password'],
    'database': config['DATABASE']['database'],
    'query_request': config['DATABASE']['query1'],
    'query_assets': config['DATABASE']['query2'],
    'query_request_with_activities': config['DATABASE']['query3'],
    'schema': config['DATABASE']['schema']
}

db_host = db_config.get('host')
db_port = db_config.get('port')
db_username = db_config.get('username')
db_password = db_config.get('password')
db_database = db_config.get('database')
db_query1 = db_config.get('query_request')
db_query2 = db_config.get('query_assets')
db_query3 = db_config.get('query_request_with_activities')
db_schema = db_config.get('schema')

In [4]:
def database_connector( db_type, host, port, database, username, password, **kwargs):
    
        encoded_password = urllib.parse.quote_plus(password)
        connection_strings = {
            'postgresql': f"postgresql://{username}:{encoded_password}@{host}:{port}/{database}",
        }
        return connection_strings[db_type]

In [5]:
conn_string = database_connector(
            db_type='postgresql',
            host=db_host,
            port=db_port,
            database=db_database,
            username=db_username,
            password=db_password,
            schema=db_schema
            )

In [6]:
last_sync_date_time = '2025-11-01 07:00:00.000'
# '2025-11-01 07:00:02.929'

In [7]:
diff_time = pd.to_datetime(last_sync_date_time) + datetime.timedelta(days=1)
upper_bound = pd.to_datetime(diff_time, format='mixed').strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]



In [8]:
upper_bound = '2025-11-02 08:00:00.000'

In [9]:
try:
        engine = create_engine(conn_string)
        
        if not db_query1:
            logger.warning("Query for v_request is missing!")
            
        else:
            db_query1 = db_query1.replace(';',f'\nWHERE "requestCreatedDate" >=\'{last_sync_date_time}\' and "requestCreatedDate" <= \'{upper_bound}\';')
    
        if not db_query2:
            logger.warning("Query for v_assets is missing!")
           
        # else:
        #     self.db_query2 = self.db_query2.replace(';',f'\nWHERE "requestCreatedDate" >=\'{self.last_sync_date_time}\';')
        
        if not db_query3:
            logger.warning("Query for v_request_with_activities is missing!")
            
        else:
            db_query3 = db_query3.replace(';',f'\nWHERE "requestCreatedDate" >=\'{last_sync_date_time}\' and "requestCreatedDate" <= \'{upper_bound}\';')

        # print(self.db_query1)
        # print(self.db_query2)
        # print(self.db_query3)

        df_request = pd.read_sql(db_query1, engine)
        df_assets = pd.read_sql(db_query2, engine)
        # df_assets = pd.read_csv('./fetched_data/v_assets.csv')
        df_request_with_activities = pd.read_sql(db_query3, engine)
        
        logger.info(f" Downloaded {len(df_request)} rows from 'v_requests', {len(df_assets)} rows from 'v_assets', and {len(df_request_with_activities)} rows from 'v_request_with_activities'.")
        
        # Saving the data (Don't save when pushing incrementally!!)
        # # self.df_request.to_csv(f"{target_dir_path}/v_requests.csv",index=False)
        # # self.df_assets.to_csv(f"{target_dir_path}/v_assets.csv",index=False)
        # # self.df_request_with_activities.to_csv(f"{target_dir_path}/v_requests_with_activities.csv",index=False)
        
        # logger.info(f"Data exported to {target_dir_path}")
        
except Exception as e:
    logger.warning(f"Error connecting to database: {e}")
    

finally:
    if 'engine' in locals():
            engine.dispose()


INFO:__main__: Downloaded 1 rows from 'v_requests', 508863 rows from 'v_assets', and 1 rows from 'v_request_with_activities'.


In [33]:
df_request

,requestId,requestAlternateId,workType,requestDescription,requestCreatedDate,requestTargetCompletionDate,requestCompletionDate,serviceClassificationId,serviceClassificationAlternateId,serviceClassificationPath,...,locationAlternateId,locationPath,assetId,assetAlternateId,assetDescription,completionNotes,customer,country,isSelfAssign,priorityCode
0,b71e6dbd-4a3f-41f2-ad82-1174c7f8c5b7,RRBC100309,Proactive,TESTING,2025-11-01 16:12:15.788,None,None,6d4a97a7-f1e4-4c1c-b37a-46a9c8cf35ee,SCRBC100235,Cleaning Services | Carpet Cleaning | Carpet -...,...,LUS508405,"US, AZ, Peoria / RBC-AZ001DR, Peoria-Sun City ...",None,None,None,None,Royal Bank of Canada,US,False,None


In [39]:
df_request_with_activities

,requestId,requestAlternateId,workType,requestDescription,requestCreatedDate,requestTargetCompletionDate,requestCompletionDate,activityId,activityAlternateId,activityDescription,activityStartDate,activityCompletionDate,completionNotes,providertype,customer,country
0,b71e6dbd-4a3f-41f2-ad82-1174c7f8c5b7,RRBC100309,Proactive,TESTING,2025-11-01 16:12:15.788,None,None,None,None,None,None,None,None,None,Royal Bank of Canada,US


In [ ]:
df_request

In [41]:
is_hvac_df_ = pd.read_csv('./data/hvac_assets/IFM_Assets_RuleBasedEngineResults(IFM_Assets_RuleBasedEngineResul).csv')
suggested_asset_df = pd.read_csv('./data/asset_suggest_data/asset_suggest_model.csv')
vendor_data = pd.read_csv('./data/asset_vendor/request_act_vendor.csv')

In [ ]:
# def data_preprocessor(self):
try:
    # Activity:
    activity_df = df_request_with_activities[df_request_with_activities['activityAlternateId'].notna()][['providertype','activityAlternateId','activityDescription']]
    activity_df.drop_duplicates(inplace = True)

    # Asset:
    requests_subset = df_request[['requestId','assetAlternateId', 'requestAlternateId']]
    requests_subset = requests_subset[requests_subset.assetAlternateId.notna()]

    v_assets = df_assets[['assetId','Asset Alt Id', 'Asset Description', 'manufacturer', 'model', 'serialNumber']]
    v_assets = v_assets.merge(requests_subset, left_on = 'Asset Alt Id', right_on = 'assetAlternateId', how= 'left')
    v_assets = v_assets[v_assets['requestAlternateId'].notna()] # keeping only those asset records which are associated to the presently fetched serviceRequests

    is_hvac_df = is_hvac_df_.copy()
    is_hvac_df['is_HVAC'] = True
    is_hvac_df.drop(columns=['Asset Description'], inplace = True)
    v_assets_with_hvac = v_assets.merge(is_hvac_df, on='Asset Alt Id', how='left')

    if not v_assets_with_hvac.empty:
        v_assets_with_hvac.loc[v_assets_with_hvac['is_HVAC'] == True, 'asset_type'] = 'HVAC'
    else:
        v_assets_with_hvac['asset_type'] = None

    final_assets_df = v_assets_with_hvac[['assetId', 'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
                                            'serialNumber', 'is_HVAC', 'asset_type', 'requestId','assetAlternateId', 'requestAlternateId']]
    final_assets_df.loc[:, 'is_HVAC'] = final_assets_df['is_HVAC'].fillna(False)

    suggested_asset_df = suggested_asset_df.copy()
    suggested_asset_df.rename(columns={'asset_id': 'suggested_asset'}, inplace=True)
    suggested_asset_df_subset = suggested_asset_df[['request_id', 'suggested_asset']]

    final_assets_df = final_assets_df.merge(suggested_asset_df_subset, left_on = 'requestId', right_on = 'request_id', how = 'left')
    final_assets_df = final_assets_df[['assetId', 'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
                                        'serialNumber', 'is_HVAC', 'asset_type', 'suggested_asset','requestAlternateId']]

    vendor_data = vendor_data.copy()
    vendor_data = vendor_data[['requestAlternateId','vendorName', 'vendorAddress1', 'vendorCity',
                        'vendorRegion', 'vendorCountry', 'vendorPostalCode']]
    assets_with_vendors = final_assets_df.merge(vendor_data, on = 'requestAlternateId', how = 'left')
    assets_df = assets_with_vendors[['Asset Description', 'Asset Alt Id', 'manufacturer', 'model','serialNumber', 
                                                'is_HVAC', 'asset_type', 'suggested_asset','vendorName', 'vendorAddress1', 'vendorCity',
                                                'vendorRegion', 'vendorCountry', 'vendorPostalCode']]

    # Country:
    country_df = df_request[['country']].drop_duplicates()

    # Customer:
    customer_df = df_request[['customer']].drop_duplicates()

    # Location:
    location_df = df_request[['locationAlternateId', 'locationPath']].drop_duplicates()

    # Service Requests:
    temp_ser_req = df_request[['isSelfAssign', 'priorityCode', 
                'requestCreatedDate', 'requestDescription', 'requestAlternateId', 'completionNotes', 
                'requestTargetCompletionDate', 'serviceClassificationAlternateId', 'serviceClassificationPath',  
                'requestCompletionDate', 'workType']]

    def to_local_datetime(date_col):

        if date_col is None:
            return None
        
        if date_col.isna().all():
            return date_col
        
        dt_series = pd.to_datetime(date_col, format='mixed')
        formatted = dt_series.dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
        
        return formatted.str.replace(' ', 'T')


    def process_service_requests(df_service_request):
            
            date_cols = ['requestCreatedDate', 'requestTargetCompletionDate', 'requestCompletionDate']
            
            for col in date_cols:
                if col in df_service_request.columns:
                    df_service_request.loc[:, col] = to_local_datetime(df_service_request[col])
            
            df_service_request['createdYear'] = pd.to_datetime(df_service_request['requestCreatedDate']).dt.year
            df_service_request['createdMonth'] = pd.to_datetime(df_service_request['requestCreatedDate']).dt.month
                
            df_service_request['isCompleted'] = df_service_request['requestCompletionDate'].notna()
            
            conditions = [
                df_service_request['requestCompletionDate'].isna(),
                df_service_request['requestTargetCompletionDate'].isna(),
                df_service_request['requestCompletionDate'] <= df_service_request['requestTargetCompletionDate'],
                df_service_request['requestCompletionDate'] > df_service_request['requestTargetCompletionDate']
            ]
            
            choices = ['Open', 'Open', 'Met', 'Miss']
            
            df_service_request['sla'] = np.select(conditions, choices, default='Unknown')
            
            return df_service_request


    service_req_df = process_service_requests(temp_ser_req)

    activity_df = activity_df.copy()
    assets_df = assets_df.copy()
    country_df = country_df.copy()
    customer_df = customer_df.copy()
    location_df = location_df.copy()
    service_req_df = service_req_df.copy()
    logger.info("Node and Property data prepared!")

except Exception as e:
    logger.warning(f"Error preprocessing data: {e}")
    traceback.print_exc()


INFO:__main__:Node and Property data prepared!


In [63]:
try:
    # renaming the features:
    activity_df.rename(columns={'activityAlternateId': 'activityId', 'providertype':'providerType'}, inplace=True)

    assets_df.rename(columns={'Asset Alt Id': 'assetId', 'Asset Description':'assetDescription', 'vendorAddress1':'vendorAddress'}, inplace=True)

    location_df.rename(columns={'locationAlternateId': 'locationId'}, inplace=True)

    service_req_df.rename(columns={'requestAlternateId': 'requestId', 'serviceClassificationAlternateId': 'serviceClassificationId'}, inplace=True)

    # logger.info(f"Data for migration to Neo4J is saved on path: {neo4j_dir_path} and ready to be imported!")

except Exception as e:
    logger.warning(f"Error Renaming Features: {e}")

In [65]:
try:

    LOCATED_AT = df_request[['assetAlternateId','locationAlternateId']].dropna().drop_duplicates()
    LOCATED_AT.rename(columns={'assetAlternateId': 'assetId', 'locationAlternateId': 'locationId'}, inplace=True)
    # self.LOCATED_AT.to_csv(f"{neo4j_relationship_dir_path}/LOCATED_AT.csv",index=False)

    AT_LOCATION = df_request[['requestAlternateId','locationAlternateId']].dropna().drop_duplicates()
    AT_LOCATION.rename(columns={'requestAlternateId': 'requestId', 'locationAlternateId': 'locationId'}, inplace=True)
    # self.AT_LOCATION.to_csv(f"{neo4j_relationship_dir_path}/AT_LOCATION.csv",index=False)

    HAS_ACTIVITY = df_request_with_activities[['activityAlternateId', 'requestAlternateId']].dropna().drop_duplicates()
    HAS_ACTIVITY.rename(columns={'requestAlternateId': 'requestId', 'activityAlternateId': 'activityId'}, inplace=True)
    # self.HAS_ACTIVITY.to_csv(f"{neo4j_relationship_dir_path}/HAS_ACTIVITY.csv",index=False)

    FOR_ASSET = df_request[['requestAlternateId','assetAlternateId']].dropna().drop_duplicates()
    FOR_ASSET.rename(columns={'requestAlternateId': 'requestId', 'assetAlternateId': 'assetId'}, inplace=True)
    # self.FOR_ASSET.to_csv(f"{neo4j_relationship_dir_path}/FOR_ASSET.csv",index=False)

    OPERATES_IN = df_request[['customer','country']].dropna().drop_duplicates()
    # self.OPERATES_IN.to_csv(f"{neo4j_relationship_dir_path}/OPERATES_IN.csv",index=False)

    RESIDES_AT = df_request[['customer','locationAlternateId']].dropna().drop_duplicates()
    RESIDES_AT.rename(columns={'locationAlternateId': 'locationId'}, inplace=True)
    # self.RESIDES_AT.to_csv(f"{neo4j_relationship_dir_path}/RESIDES_AT.csv",index=False)

    OWNS = df_request[['customer','assetAlternateId']].dropna().drop_duplicates()
    OWNS.rename(columns={'assetAlternateId': 'assetId'}, inplace=True)
    # self.OWNS.to_csv(f"{neo4j_relationship_dir_path}/OWNS.csv",index=False)

    CREATES = df_request[['customer','requestAlternateId']].dropna().drop_duplicates()
    CREATES.rename(columns={'requestAlternateId': 'requestId'}, inplace=True)
    # self.CREATES.to_csv(f"{neo4j_relationship_dir_path}/CREATES.csv",index=False)

    IN = df_request[['country','locationAlternateId']].dropna().drop_duplicates()
    IN.rename(columns={'locationAlternateId': 'locationId'}, inplace=True)
    # self.IN.to_csv(f"{neo4j_relationship_dir_path}/IN.csv",index=False)

    logger.info(f"Relationships created!")

except Exception as e:
    logger.warning(f"Error while creating and saving relationships: {e}")



INFO:__main__:Relationships created!


In [78]:
if not Path(config_file_path).exists():
    logger.warning(f"Config file {config_file_path} not found!")

config = configparser.ConfigParser()
config.read(config_file_path)

if 'Neo4j' not in config:
    raise ValueError("Neo4j section not found in config")

nj_config = {
    'url': config['Neo4j']['url'],
    'username': config['Neo4j']['username'],
    'password': config['Neo4j']['password']
}

nj_url = nj_config.get('url')
nj_username = nj_config.get('username')
nj_password = nj_config.get('password')

driver = GraphDatabase.driver(nj_url, auth=(nj_username, nj_password))

In [79]:
def load_nodes_from_csv( csv, node_label: str, id_property: str, batch_size: int = 1000):
        """Load nodes from CSV file in batches."""
        df = csv
        df = df.where(pd.notnull(df), None)  # Replace NaN with None
        
        total_rows = len(df)
        # logger.info(f"Loading {total_rows} {node_label} nodes from {csv_path}")
        
        with driver.session() as session:
            for i in range(0, total_rows, batch_size):
                batch = df.iloc[i:i+batch_size]
                records = batch.to_dict('records')
                
                # Build Cypher query dynamically
                query = f"""
                UNWIND $records AS record
                MERGE (n:{node_label} {{{id_property}: record.{id_property}}})
                SET n += record
                """
                session.run(query, records=records)
                logger.info(f"Loaded batch {i//batch_size + 1}/{(total_rows-1)//batch_size + 1} for {node_label}")
        
        logger.info(f"Completed loading {node_label} nodes")


In [80]:
try:

    # # Create constraints and indexes              # not required when hydrating data to a preexisting graph
    # self.create_constraints()
    # self.create_indexes()
    
    # Load nodes
    
    load_nodes_from_csv(
        activity_df, 
        "Activity", 
        "activityId"
    )
    
    load_nodes_from_csv(
        assets_df, 
        "Asset", 
        "assetId"
    )
    
    load_nodes_from_csv(
        country_df, 
        "Country", 
        "country"
    )
    
    load_nodes_from_csv(
        customer_df, 
        "Customer", 
        "customer"
    )
    
    load_nodes_from_csv(
        location_df, 
        "Location", 
        "locationId"
    )
    
    load_nodes_from_csv(
        service_req_df, 
        "ServiceRequest", 
        "requestId"
    )

except Exception as e:
    logger.warning(f"Error creating Nodes in Neo4j: {e}")


INFO:__main__:Completed loading Activity nodes
INFO:__main__:Completed loading Asset nodes
INFO:__main__:Loaded batch 1/1 for Country
INFO:__main__:Completed loading Country nodes
INFO:__main__:Loaded batch 1/1 for Customer
INFO:__main__:Completed loading Customer nodes
INFO:__main__:Loaded batch 1/1 for Location
INFO:__main__:Completed loading Location nodes
INFO:__main__:Loaded batch 1/1 for ServiceRequest
INFO:__main__:Completed loading ServiceRequest nodes


In [81]:

def load_relationships_from_csv(csv, rel_config: Dict, batch_size: int = 1000):
    """
    Load relationships from CSV file.
    
    rel_config example:
    {
        'rel_type': 'LOCATED_AT',
        'from_label': 'Asset',
        'from_id_col': 'assetId',
        'from_id_prop': 'assetId',
        'to_label': 'Location',
        'to_id_col': 'locationId',
        'to_id_prop': 'locationId',
        'properties': []  # Optional: list of relationship properties
    }
    """
    df = csv
    df = df.where(pd.notnull(df), None)
    
    total_rows = len(df)
    # logger.info(f"Loading {total_rows} {rel_config['rel_type']} relationships from {csv_path}")
    
    with driver.session() as session:
        for i in range(0, total_rows, batch_size):
            batch = df.iloc[i:i+batch_size]
            records = batch.to_dict('records')
            
            # Build relationship properties string if any
            rel_props = ""
            if rel_config.get('properties'):
                props_str = ", ".join([f"{p}: record.{p}" for p in rel_config['properties']])
                rel_props = f" {{{props_str}}}"
            
            query = f"""
            UNWIND $records AS record
            MATCH (from:{rel_config['from_label']} {{{rel_config['from_id_prop']}: record.{rel_config['from_id_col']}}})
            MATCH (to:{rel_config['to_label']} {{{rel_config['to_id_prop']}: record.{rel_config['to_id_col']}}})
            MERGE (from)-[r:{rel_config['rel_type']}]->(to)
            """
            
            if rel_props:
                query += f"\nSET r += {{{', '.join([f'{p}: record.{p}' for p in rel_config['properties']])}}}"
            
            session.run(query, records=records)
            logger.info(f"Loaded batch {i//batch_size + 1}/{(total_rows-1)//batch_size + 1} for {rel_config['rel_type']}")
    
    logger.info(f"Completed loading {rel_config['rel_type']} relationships")


In [82]:
try:
    # Load relationships
    
    # Asset -> Location
    load_relationships_from_csv(
        LOCATED_AT,
        {
            'rel_type': 'LOCATED_AT',
            'from_label': 'Asset',
            'from_id_col': 'assetId',
            'from_id_prop': 'assetId',
            'to_label': 'Location',
            'to_id_col': 'locationId',
            'to_id_prop': 'locationId'
        }
    )
    
    # ServiceRequest -> Location
    load_relationships_from_csv(
        AT_LOCATION,
        {
            'rel_type': 'AT_LOCATION',
            'from_label': 'ServiceRequest',
            'from_id_col': 'requestId',
            'from_id_prop': 'requestId',
            'to_label': 'Location',
            'to_id_col': 'locationId',
            'to_id_prop': 'locationId'
        }
    )
    
    # ServiceRequest -> Activity
    load_relationships_from_csv(
        HAS_ACTIVITY,
        {
            'rel_type': 'HAS_ACTIVITY',
            'from_label': 'ServiceRequest',
            'from_id_col': 'requestId',
            'from_id_prop': 'requestId',
            'to_label': 'Activity',
            'to_id_col': 'activityId',
            'to_id_prop': 'activityId'
        }
    )
    
    # ServiceRequest -> Asset
    load_relationships_from_csv(
        FOR_ASSET,
        {
            'rel_type': 'FOR_ASSET',
            'from_label': 'ServiceRequest',
            'from_id_col': 'requestId',
            'from_id_prop': 'requestId',
            'to_label': 'Asset',
            'to_id_col': 'assetId',
            'to_id_prop': 'assetId'
        }
    )
    
    # Customer -> Country
    load_relationships_from_csv(
        OPERATES_IN,
        {
            'rel_type': 'OPERATES_IN',
            'from_label': 'Customer',
            'from_id_col': 'customer',
            'from_id_prop': 'customer',
            'to_label': 'Country',
            'to_id_col': 'country',
            'to_id_prop': 'country'
        }
    )
    
    # Customer -> Location
    load_relationships_from_csv(
        RESIDES_AT,
        {
            'rel_type': 'RESIDES_AT',
            'from_label': 'Customer',
            'from_id_col': 'customer',
            'from_id_prop': 'customer',
            'to_label': 'Location',
            'to_id_col': 'locationId',
            'to_id_prop': 'locationId'
        }
    )
    
    # Customer -> Asset
    load_relationships_from_csv(
        OWNS,
        {
            'rel_type': 'OWNS',
            'from_label': 'Customer',
            'from_id_col': 'customer',
            'from_id_prop': 'customer',
            'to_label': 'Asset',
            'to_id_col': 'assetId',
            'to_id_prop': 'assetId'
        }
    )
    
    # Customer -> ServiceRequest
    load_relationships_from_csv(
        CREATES,
        {
            'rel_type': 'CREATES',
            'from_label': 'Customer',
            'from_id_col': 'customer',
            'from_id_prop': 'customer',
            'to_label': 'ServiceRequest',
            'to_id_col': 'requestId',
            'to_id_prop': 'requestId'
        }
    )
    
    # Location -> Country
    load_relationships_from_csv(
        IN,
        {
            'rel_type': 'IN',
            'from_label': 'Location',
            'from_id_col': 'locationId',
            'from_id_prop': 'locationId',
            'to_label': 'Country',
            'to_id_col': 'country',
            'to_id_prop': 'country'
        }
    )

except Exception as e:
    logger.warning(f"Error creating Relationships for nodes on Neo4j: {e}")


INFO:__main__:Completed loading LOCATED_AT relationships
INFO:__main__:Loaded batch 1/1 for AT_LOCATION
INFO:__main__:Completed loading AT_LOCATION relationships
INFO:__main__:Completed loading HAS_ACTIVITY relationships
INFO:__main__:Completed loading FOR_ASSET relationships
INFO:__main__:Loaded batch 1/1 for OPERATES_IN
INFO:__main__:Completed loading OPERATES_IN relationships
INFO:__main__:Loaded batch 1/1 for RESIDES_AT
INFO:__main__:Completed loading RESIDES_AT relationships
INFO:__main__:Completed loading OWNS relationships
INFO:__main__:Loaded batch 1/1 for CREATES
INFO:__main__:Completed loading CREATES relationships
INFO:__main__:Loaded batch 1/1 for IN
INFO:__main__:Completed loading IN relationships


In [ ]:
def close(self):
    self.driver.close()

Bad pipe message: %s [b'\x9c6\xaf\xf4\xd7T]M,\xc5\x8etZ\xb7\x01\xa1\x0c\x1f\x00\x01|\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00@\x00A\x00B\x00C\x00D\x00E\x00F\x00g\x00h\x00i\x00j\x00k\x00l\x00m\x00\x84\x00\x85\x00\x86\x00\x87\x00\x88\x00\x89\x00\x96\x00\x97\x00\x98\x00\x99\x00\x9a\x00\x9b\x00\x9c\x00\x9d\x00\x9e\x00\x9f\x00\xa0\x00\xa1\x00\xa2\x00\xa3\x00\xa4\x00\xa5\x00\xa6\x00\xa7\x00\xba\x00\xbb\x00\xbc\x00\xbd\x00\xbe\x00\xbf\x00\xc0\x00\xc1\x00\xc2\x00']
Bad pipe message: %s [b"44\xb2c\xc9\x1c]'U\xe9\xb9\x06\xc0\xa7\xe8\x8do\x8a\x00\x01|\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x